to do: add api call so I don't have to worry about the size of the data

# Capstone Project

## Project Definition

Investistigate the American Time Use Survey (ATUS) and Predict something tbd


## Analysis

Inspiration https://github.com/hwangmpaula/data-wrangling/blob/master/wrangle_act.ipynb

### Gather

Using the ATUS data from https://www.kaggle.com/bls/american-time-use-survey I will analyse whether we see an increase in active leisure vs passive leisure with an increase in authority in the roles. 

The data forms I need are the **Activity summary** file and the **Respondent** file.

**Activity summary** from Kaggle:

> The Activity summary file contains information about the total time each ATUS respondent spent doing each activity on the diary day.

**Respondent** from Kaggle:

> The Respondent file contains information about ATUS respondents, including their labor force status and earnings.

These are already packaged in neat .csvs for data science use.

In [10]:
#import libraries
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# Load datasets
ATUS_sum = pd.read_csv('../data/atussum.csv') #The activity summary datafile
ATUS_resp = pd.read_csv('../data/atusresp.csv') #The respondent datafile

### Assess

**1. Quality**
(0 issues)
  - issue here

**2. Tidyness**
(2 issues)
   - Remove unnecessary rows
   - Remove unnecessary columns
   - Unify into one dataframe

In [12]:
# A quick visual of the activity summary dataframe
ATUS_sum.head(10)

,tucaseid,gemetsta,gtmetsta,peeduca,pehspnon,ptdtrace,teage,telfs,temjot,teschenr,...,t181801,t181899,t189999,t500101,t500103,t500104,t500105,t500106,t500107,t509989
0,20030100013280,1,-1,44,2,2,60,2,2,-1,...,0,0,0,0,0,0,0,0,0,0
1,20030100013344,2,-1,40,2,1,41,1,2,2,...,0,0,0,0,0,0,0,0,0,0
2,20030100013352,1,-1,41,2,1,26,2,2,2,...,0,0,0,0,0,0,0,0,0,0
3,20030100013848,2,-1,39,2,2,36,4,-1,2,...,0,0,0,0,0,0,0,0,0,0
4,20030100014165,2,-1,45,2,1,51,1,2,-1,...,0,0,0,0,0,0,0,0,0,0
5,20030100014169,2,-1,43,2,1,32,2,2,1,...,0,0,0,0,0,0,0,0,0,0
6,20030100014209,1,-1,39,2,1,44,1,2,2,...,0,0,0,0,0,0,0,0,0,0
7,20030100014427,1,-1,40,2,1,21,1,2,2,...,0,0,0,0,0,0,0,0,0,0
8,20030100014550,2,-1,41,2,1,33,1,2,2,...,0,0,0,0,0,0,0,0,0,0
9,20030100014758,1,-1,41,2,2,39,1,2,2,...,0,0,0,0,0,0,0,0,0,0


In [18]:
# Get the shape of the activity summary data
ATUS_sum.shape

(170842, 455)

In [19]:
# Check the case ids for uniqueness
ATUS_sum.tucaseid.nunique()

170842

In [32]:
# Looks good. Now to figure out which is the first column with activity information
ATUS_sum_columns = ATUS_sum.columns.to_list()
ATUS_sum_columns[:30]

['tucaseid',
 'gemetsta',
 'gtmetsta',
 'peeduca',
 'pehspnon',
 'ptdtrace',
 'teage',
 'telfs',
 'temjot',
 'teschenr',
 'teschlvl',
 'tesex',
 'tespempnot',
 'trchildnum',
 'trdpftpt',
 'trernwa',
 'trholiday',
 'trspftpt',
 'trsppres',
 'tryhhchild',
 'tudiaryday',
 'tufnwgtp',
 'tehruslt',
 'tuyear',
 't010101',
 't010102',
 't010199',
 't010201',
 't010299',
 't010301']

In [33]:
# 't010101' is the first activity column (which matches the data dictionary)
# Setting it's location in a variable for later
act_start = ATUS_sum_columns.index('t010101')

In [39]:
# Checking for any NaN values in the data I'll be using
ATUS_sum[ATUS_sum_columns[act_start:]].isnull().sum().sort_values(ascending=False)

t509989    0
t060101    0
t060103    0
t060104    0
t060199    0
          ..
t130136    0
t130199    0
t130201    0
t130202    0
t010101    0
Length: 431, dtype: int64

In [45]:
# No NaNs, good sign.
# 
ATUS_sum['t010101'].value_counts()

480     10495
540     10296
510      8708
600      7467
450      7225
        ...  
1088        1
961         1
1089        1
1345        1
1023        1
Name: t010101, Length: 1120, dtype: int64

In [13]:
# A quick visual of the respondent dataframe
ATUS_resp.head(10)

,tucaseid,tulineno,tespuhrs,trdtind1,trdtocc1,trernhly,trernupd,trhernal,trhhchild,trimind1,...,tryhhchild,trwbmodr,trtalone_wk,trtccc_wk,trlvmodr,trtec,tuecytd,tuelder,tuelfreq,tuelnum
0,20030100013280,1,-1.0,40,8,2200.0,1,1,2,15,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
1,20030100013344,1,50.0,16,16,-1.0,1,-1,1,5,...,0,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,20030100013352,1,-1.0,43,15,1250.0,0,0,2,16,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
3,20030100013848,1,40.0,-1,-1,-1.0,-1,-1,1,-1,...,9,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,20030100014165,1,-1.0,42,10,-1.0,-1,-1,1,16,...,14,-1,-1,-1,-1,-1,-1,-1,-1,-1
5,20030100014169,1,40.0,40,8,-1.0,1,-1,1,15,...,2,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,20030100014209,1,50.0,43,15,-1.0,-1,-1,1,16,...,9,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,20030100014427,1,-1.0,41,11,950.0,0,0,1,16,...,14,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,20030100014550,1,40.0,34,17,1400.0,0,0,1,12,...,3,-1,-1,-1,-1,-1,-1,-1,-1,-1
9,20030100014758,1,-1.0,41,11,1200.0,0,0,1,16,...,4,-1,-1,-1,-1,-1,-1,-1,-1,-1


In [20]:
#get the shape of the respondent data
ATUS_resp.shape

(170842, 132)

In [21]:
#check the case ids for uniqueness
ATUS_resp.tucaseid.nunique()

170842

### Clean

Method for each assess issue

 - Define the problem
 - Code the solution
 - Test the solution

**Remove unnecessary columns in Activity summary - define**

For **Activity summary** I do not need any columns that contain data on the respondant, I only need to know what their time spent doing the individual 

### Analyze

### Model

inspired by Data Driven Science and Engineering by Steve L. Brunton and J. Nathan Kutz 
http://www.databookuw.com/

In [ ]:
U, S, VT = np.linalg.svd(A,full_matrices=0)
x = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ b

fig = plt.figure()
ax1 = fig.add_subplot(121)

A_mean = np.mean(A,axis=0)
A_mean = A_mean.reshape(-1, 1)

A2 = A - np.ones((A.shape[0],1)) @ A_mean.T

for j in range(A.shape[1]-1):
    A2std = np.std(A2[:,j])
    A2[:,j] = A2[:,j]/A2std
    
A2[:,-1] = np.ones(A.shape[0])

U, S, VT = np.linalg.svd(A2,full_matrices=0)
x = VT.T @ np.linalg.inv(np.diag(S)) @ U.T @ b

### Visualize

inspired by Data Driven Science and Engineering by Steve L. Brunton and J. Nathan Kutz 
http://www.databookuw.com/

In [2]:
plt.plot(b, Color='k', LineWidth=2, label='Housing Value') # True relationship
plt.plot(A@x, '-o', Color='r', LineWidth=1.5, MarkerSize=6, label='Regression')
plt.xlabel('Neighborhood')
plt.ylabel('Median Home Value [$1k]')
plt.legend()

ax2 = fig.add_subplot(122)
sort_ind = np.argsort(H[:,-1])
b = b[sort_ind] # sorted values
plt.plot(b, Color='k', LineWidth=2, label='Housing Value') # True relationship
plt.plot(A[sort_ind,:]@x, '-o', Color='r', LineWidth=1.5, MarkerSize=6, label='Regression')
plt.xlabel('Neighborhood')
plt.legend()

plt.show()

NameError: name 'b' is not defined

In [ ]:
x_tick = range(len(x)-1)+np.ones(len(x)-1)
plt.bar(x_tick,x[:-1])
plt.xlabel('Attribute')
plt.ylabel('Significance')
plt.xticks(x_tick)
plt.show()